In [11]:
from torchvision import transforms
from PIL import Image
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

image = Image.open('../dataset_2d/proclined/proclined_1.png')

transformed_image = transform(image)
transformed_image.shape


torch.Size([1, 1250, 1206])